In [ ]:
"""
import random, string
password = ''.join(random.choice(string.ascii_letters + string.digits) for i in range(20))

#Download ngrok
! wget -q -c -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip -qq -n ngrok-stable-linux-amd64.zip
#Setup sshd
! apt-get install -qq -o=Dpkg::Use-Pty=0 openssh-server pwgen > /dev/null
#Set root password
! echo root:$password | chpasswd
! mkdir -p /var/run/sshd
! echo "PermitRootLogin yes" >> /etc/ssh/sshd_config
! echo "PasswordAuthentication yes" >> /etc/ssh/sshd_config
! echo "LD_LIBRARY_PATH=/usr/lib64-nvidia" >> /root/.bashrc
! echo "export LD_LIBRARY_PATH" >> /root/.bashrc

#Run sshd
get_ipython().system_raw('/usr/sbin/sshd -D &')

#Ask token
print("Copy authtoken from https://dashboard.ngrok.com/auth")
import getpass
authtoken = getpass.getpass()

#Create tunnel
get_ipython().system_raw('./ngrok authtoken $authtoken && ./ngrok tcp 22 &')
#Print root password
print("Root password: {}".format(password))
#Get public address
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

"""

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.initialize_tpu_system(tpu)
  tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

In [ ]:
# multivariate multi-step stacked lstm example
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
import matplotlib.pyplot as plt
from pandas import read_csv
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from keras.layers.core import Dense, Activation, Dropout
from datetime import datetime
import time
from keras.activations import *
import tensorflow as tf
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
#sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
scaler  = MinMaxScaler()
df_train =0
df_test  =0
carpan =1
name   = "M1_hazirVeri_train_1March_dongulu"
dbname = "M1_dataframes_dongulu"

"""def TimeStamps(dataframe_Date):
    dTime = dataframe_Date
    if name == ('Weekly'or 'Daily' or 'Monthly'):
        return pd.DataFrame(array([datetime.strptime((dTime[i]),'%Y %m %d').timestamp() for i in range(len(dTime))]))#%H:%M:%S
    else :
        return pd.DataFrame(array([datetime.strptime((dTime[i]),'%Y %m %d %H %M %S').timestamp() for i in range(len(dTime))]))
"""
import dill
try:
    """
    df_train = read _csv('/content/drive/Othercomputers/2022 Masaüstü 2/Dersler 2022/Dersler-2021-2022-Donem-2/Lisans Github/EURUSD_'+name+'.csv', engine='python')
    df_test  = read_csv('/content/drive/Othercomputers/2022 Masaüstü 2/Dersler 2022/Dersler-2021-2022-Donem-2/Lisans Github/EURUSD_'+name+'.csv', engine='python')
    #df_train['Date'] = TimeStamps(df_train['Date'])
    #df_test['Date']  = TimeStamps(df_test['Date'])
    dill.dump_session('/content/drive/Othercomputers/2022 Masaüstü 2/Dersler 2022/Dersler-2021-2022-Donem-2/Lisans Github/'+dbname+'.db')
    """
    dill.load_session('/content/drive/Othercomputers/2022 Masaüstü 2/Dersler 2022/Dersler-2021-2022-Donem-2/Lisans Github/'+dbname+'.db')
    
except:
    """
    df_train = read_csv('C:/Users/ertad/Dersler 2022/Dersler-2021-2022-Donem-2/Lisans Github/EURUSD_'+name+'.csv', engine='python')
    df_test  = read_csv('C:/Users/ertad/Dersler 2022/Dersler-2021-2022-Donem-2/Lisans Github/EURUSD_'+name+'.csv' , engine='python')
    #df_train['Date'] = TimeStamps(df_train['Date'])
    #df_test['Date']  = TimeStamps(df_test['Date'])
    dill.dump_session('C:/Users/ertad/Dersler 2022/Dersler-2021-2022-Donem-2/Lisans Github/'+dbname+'.db')
    """
    dill.load_session('C:/Users/ertad/Dersler 2022/Dersler-2021-2022-Donem-2/Lisans Github/'+dbname+'.db')
    
df_train,df_test

In [2]:
sin = df_train['month_sin']
cos = df_train['month_cos']
sin = df_train['weekly_sin'] + sin
cos = df_train['weekly_cos'] + cos
sin = df_train['day_sin'] + sin
cos = df_train['day_cos'] + cos
sin = df_train['hour_sin'] + sin
cos = df_train['hour_cos'] + cos
sin = df_train['minute_sin'] + sin
cos = df_train['minute_cos'] + cos

date_train =  pd.concat([sin,cos,df_train['Open']],axis=1)

sin = df_test['month_sin']
cos = df_test['month_cos']
sin = df_test['weekly_sin'] + sin
cos = df_test['weekly_cos'] + cos
sin = df_test['day_sin'] + sin
cos = df_test['day_cos'] + cos
sin = df_test['hour_sin'] + sin
cos = df_test['hour_cos'] + cos
sin = df_test['minute_sin'] + sin
cos = df_test['minute_cos'] + cos

date_test =  pd.concat([sin,cos,df_test['Open']],axis=1)

date_train,date_test

(               0        1     Open
 0        0.87963  4.83561  1.00790
 1        1.07846  4.77101  1.00780
 2        1.62278  4.16648  1.00790
 3        1.64987  4.06537  1.00800
 4        1.66624  3.96199  1.00780
 ...          ...      ...      ...
 7989910 -0.19345  4.74901  1.13245
 7989911 -0.09573  4.78652  1.13253
 7989912  0.00538  4.81361  1.13253
 7989913  0.10876  4.82998  1.13253
 7989914  0.48700  4.86091  1.13244
 
 [7989915 rows x 3 columns],              0        1     Open
 0      1.25951  4.66648  1.13689
 1      1.34085  4.60060  1.13655
 2      1.41486  4.52659  1.13678
 3      1.48074  4.44525  1.13674
 4      1.53775  4.35746  1.13674
 ...        ...      ...      ...
 56081  1.42282  3.86622  1.11922
 56082  1.52054  3.90373  1.11929
 56083  1.62165  3.93082  1.11935
 56084  1.72503  3.94719  1.11932
 56085  2.10407  3.97971  1.11908
 
 [56086 rows x 3 columns])

In [15]:
model = Sequential()
giris = 3
cikis = 3
def SetModel(units,train,dense,activations='relu',optimizer='adam', loss='mse'):
    print("Set Mode train shape 0 : ",train.shape[0])
    model.add(LSTM(units, activation=activations,input_shape=(train.shape[1],giris)))
    """
    #model.add(Dropout(0.2))
    model.add(Dense(4))
    model.add(Dense(4))
    model.add(Dense(4))
    model.add(Dense(4))
    #model.add(Dense(512, activation=activations))
    model.add(Dense(dense))
    model.compile(optimizer, loss)
    """
    #model.add(Dense(256*4))
    #model.add(Dropout(0.2))
    #model.add(Dense(256*2))
    #model.add(Dropout(0.2))
    #model.add(Dense(256))
    #model.add(Dropout(0.2))
    #model.add(Dense(256))
    model.add(Dropout(0.2))
    model.add(Dense(128))
    model.add(Dropout(0.5))
    model.add(Dense(64))
    model.add(Dense(cikis))
    opt = tf.keras.optimizers.SGD(learning_rate=0.00001)
    model.compile(optimizer='adam', loss='mae', metrics=['accuracy'])
    print(model.summary())
    
def setupDataFrame(dataFrame,train_Dataframe):
    #X
    X_Data = dataFrame
    #print("Orjinal Shape X :",X_Data.shape)
    X_Data = np.array(X_Data)
    X_Data[~np.isnan(X_Data).any(axis=1)]

    X_Data= X_Data.reshape((len(X_Data),1,giris))
    #print("Yeni Shape X :",X_Data.shape)
    #X_Data = scaler.fit_transform(X_Data)#.round(5)

    #Y
    Y_Data = train_Dataframe
    Y_Data[~np.isnan(Y_Data).any(axis=1)]
    Y_Data =  pd.concat([Y_Data['High'],Y_Data['Low'],Y_Data['Close']],axis=1)
    Y_Data = (Y_Data*carpan).values#.round(5)
    #print("Orjinal Shape Y :",Y_Data.shape)
    Y_Data = np.array(Y_Data).reshape((len(Y_Data),1,cikis))
    #Y_Data = scaler.fit_transform(Y_Data)#.round(5)
    #print("Yeni Shape Y :",Y_Data.shape)
    #print(X_t,y_t)
    return X_Data,Y_Data

def FUNC_Time(train_dataframe,test_dataframe):
    #X_train #Y_train
    X_train,y_train =setupDataFrame(date_train,train_dataframe)
    #X_test #y_test
    X_test,y_test = setupDataFrame(date_test,test_dataframe)
    #SetModel
    SetModel(units=6,train=X_train,dense=cikis)
    print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)
    print(X_train,"\n",y_train,"\n",X_test,"\n",y_test,"\n")
    

    return X_train,y_train,X_test,y_test

X_train,y_train,X_test,y_test = FUNC_Time(df_train,df_test)

Set Mode train shape 0 :  7989915
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 6)                 240       
                                                                 
 dropout_2 (Dropout)         (None, 6)                 0         
                                                                 
 dense_2 (Dense)             (None, 128)               896       
                                                                 
 dropout_3 (Dropout)         (None, 128)               0         
                                                                 
 dense_3 (Dense)             (None, 64)                8256      
                                                                 
 dense_4 (Dense)             (None, 3)                 195       
                                                                 
Total params: 9,587


In [17]:
#for i in range(100):
history = model.fit(X_train,y_train, epochs=15,batch_size=1024*3,validation_data=(X_test,y_test))  
predict = model.predict(X_test)#.round(5)
print(predict,y_test)


Epoch 1/15
2601/2601 [==============================] - 136s 52ms/step - loss: 0.1264 - accuracy: 0.3383 - val_loss: 0.0668 - val_accuracy: 0.0000e+00
Epoch 2/15
2601/2601 [==============================] - 128s 49ms/step - loss: 0.1262 - accuracy: 0.6384 - val_loss: 0.0795 - val_accuracy: 1.0000
Epoch 3/15
2601/2601 [==============================] - 128s 49ms/step - loss: 0.1261 - accuracy: 0.8044 - val_loss: 0.0754 - val_accuracy: 1.0000
Epoch 4/15
2601/2601 [==============================] - 129s 49ms/step - loss: 0.1261 - accuracy: 0.7602 - val_loss: 0.0672 - val_accuracy: 1.0000
Epoch 5/15
2601/2601 [==============================] - 129s 50ms/step - loss: 0.1261 - accuracy: 0.7564 - val_loss: 0.0764 - val_accuracy: 0.0000e+00
Epoch 6/15
2601/2601 [==============================] - 130s 50ms/step - loss: 0.1261 - accuracy: 0.7219 - val_loss: 0.0704 - val_accuracy: 1.0000
Epoch 7/15
2601/2601 [==============================] - 128s 49ms/step - loss: 0.1261 - accuracy: 0.7213 - val

h1 units 16 1.8535e-06
h1 units 32 1.9282e-06

In [13]:
predict = model.predict(X_test[-50:])#.round(5)

In [14]:
predict,"\n",y_test

(array([[1.2120472, 1.2117658, 1.2117555],
        [1.2120475, 1.2117656, 1.2117555],
        [1.2120476, 1.2117655, 1.2117556],
        [1.2120477, 1.2117654, 1.2117558],
        [1.2120478, 1.2117653, 1.2117558],
        [1.212048 , 1.211765 , 1.2117559],
        [1.2120482, 1.2117649, 1.211756 ],
        [1.2120484, 1.2117648, 1.2117561],
        [1.2120485, 1.2117646, 1.2117561],
        [1.2120486, 1.2117645, 1.2117562],
        [1.2120489, 1.2117643, 1.2117563],
        [1.212049 , 1.2117642, 1.2117565],
        [1.2120491, 1.211764 , 1.2117565],
        [1.2120494, 1.2117639, 1.2117566],
        [1.2120495, 1.2117637, 1.2117567],
        [1.2120496, 1.2117636, 1.2117567],
        [1.2120497, 1.2117635, 1.2117568],
        [1.2120498, 1.2117635, 1.2117568],
        [1.2120498, 1.2117634, 1.211757 ],
        [1.21205  , 1.2117633, 1.211757 ],
        [1.21205  , 1.2117633, 1.211757 ],
        [1.21205  , 1.2117633, 1.211757 ],
        [1.2120501, 1.2117633, 1.211757 ],
        [1.

In [ ]:
#test= X_test.reshape(56086,2)
#yy   = y_test.reshape(56086,3)
#pre = np.array(predict).reshape(56086,3)
#plt.plot(test[0],[yy[0],yy[1],yy[2]],color='blue')
#plt.plot(test[0],[pre[0],pre[1],pre[2]],color='red')

In [ ]:
yeni = np.array([1.09160])

model.predict(yeni.reshape(-1,1))#.round(5)

In [ ]:
#from tensorflow.python.client import device_lib
#import tensorflow as tf
#tf.debugging.set_log_device_placement(True)
#print(device_lib.list_local_devices())